<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/077_Handling_Session_Specific_Dependencies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Handling Session or Request-Specific Dependencies

By using `ActionContext`, we’ve solved several key challenges:

* ✅ Tools can access conversation history without being coupled to the agent implementation
* ✅ Authentication and other request-specific information can be injected where needed
* ✅ Tools remain independent and testable since their dependencies are explicitly declared
* ✅ The agent can provide different contexts for different execution environments (development, production, testing)

This pattern of dependency injection through `ActionContext` becomes even more valuable when we need to handle authentication-specific information.

---

### Example Tool: Authenticated API Update

Consider a tool that needs to update a project management system with the status of code reviews. This requires authentication, but we don’t want to hardcode credentials or configure them at the tool level:

```python
@register_tool(
    description="Update code review status in project management system",
    tags=["project_management"]
)
def update_review_status(action_context: ActionContext,
                         review_id: str,
                         status: str) -> dict:
    """Update the status of a code review in the project system."""
    # Get the authentication token for this specific request
    auth_token = action_context.get("auth_token")
    if not auth_token:
        raise ValueError("Authentication token not found in context")
    
    # Make authenticated request
    headers = {
        "Authorization": f"Bearer {auth_token}",
        "Content-Type": "application/json"
    }
    
    response = requests.post(
        f"https://...someapi.../reviews/{review_id}/status",
        headers=headers,
        json={"status": status}
    )
    
    if response.status_code != 200:
        raise ValueError(f"Failed to update review status: {response.text}")
        
    return {"status": "updated", "review_id": review_id}
```

---

### When the Agent Runs a Tool

The agent provides all necessary context through the `ActionContext`:

```python
def run(self, user_input: str, memory=None, action_context_props=None):
    """Execute the agent loop."""
    memory = memory or Memory()
    
    # Create context with all necessary resources
    action_context = ActionContext({
        'memory': memory,
        'llm': self.generate_response,
        # Request-specific auth
        **action_context_props
    })
    
    while True:
        prompt = self.construct_prompt(action_context, self.goals, memory)
        response = self.prompt_llm_for_action(action_context, prompt)
        result = self.handle_agent_response(action_context, response)
        
        if self.should_terminate(action_context, response):
            break
```

---

### Running the Agent with Specific Dependencies

```python
# Run the agent and create custom context for the action to
# pass to tools that need it
some_agent.run("Update the project status...",
               memory=...,
               # Pass request-specific auth token
               action_context_props={"auth_token": "my_auth_token"})
```






## ✅ Key Features

### 1. **Use of `ActionContext`**

This object is central to how tools get their dependencies (like memory, LLM, or auth tokens). You're seeing it used here to:

* Inject `memory`, `llm`, and `auth_token`
* Keep tools decoupled from the agent logic
* Allow session- or request-specific values (like a new token per user request)

This is a **flexible and testable pattern** that allows for modularity.

---

### 2. **Tool Receives Only What It Needs**

In the `update_review_status` tool:

```python
def update_review_status(action_context: ActionContext, review_id: str, status: str)
```

The tool:

* Explicitly asks for `action_context`
* Pulls only `auth_token` from it
* Performs a POST request using the token

This is **good practice** because:

* It avoids hardcoding sensitive data
* Keeps tools reusable and secure

---

### 3. **Context Merging with `action_context_props`**

In the `agent.run(...)` function:

```python
action_context = ActionContext({
    'memory': memory,
    'llm': self.generate_response,
    **action_context_props
})
```

Here:

* `**action_context_props` dynamically injects session/request-specific data like `auth_token`
* This is **scoped configuration** — meaning it's injected only when needed

This design makes your agent reusable across sessions and contexts.

---

### 4. **Agent Loop with Tool Execution**

The loop:

```python
while True:
    ...
    result = self.handle_agent_response(action_context, response)
```

This shows:

* Continuous interaction with tools
* Separation of decision-making (by the agent) and action execution (via tools)

It’s a clean architecture where **the agent decides**, but **the environment/tool handles execution**.

---

## 🎯 What You Should Be Focusing On

| Focus Area                                                                     | Why It Matters                                                          |
| ------------------------------------------------------------------------------ | ----------------------------------------------------------------------- |
| ✅ **How `ActionContext` is created and used**                                  | It’s the hub of all your injected dependencies                          |
| ✅ **How `action_context_props` lets you inject session/request-specific data** | This is what makes your agents flexible and secure                      |
| ✅ **The separation of concerns**                                               | The agent doesn't manage auth or memory directly — the environment does |
| ✅ **The fact that tools don’t “know” about each other or the agent**           | This keeps the system modular, testable, and maintainable               |
| ✅ **Security and minimal sharing**                                             | The tool gets just the token it needs, and nothing more                 |






### ✅ Key Takeaways Before Moving On

1. **This pattern scales beautifully.**
   You can now reuse the same agent architecture across different users, teams, and environments just by changing the `action_context_props`.

2. **Testability is built-in.**
   Since tools don’t rely on global state or hardcoded values, you can test each one in isolation by passing in a mock `ActionContext`.

3. **Security is improved.**
   Tools only get the dependencies they declare. That means you avoid leaking things like auth tokens to tools that don’t need them.

4. **This is real-world engineering.**
   What you’re learning isn’t academic. This kind of design (dependency injection, scoped config, context objects) is used in production-grade systems at scale.

---

If you understand how:

* `ActionContext` wraps context
* `action_context_props` customizes context per run
* The **agent decides**, and the **tool acts** using dependencies

...then you're ready for the next level.


